In [14]:
# @title Environment Setup
import os
import sys

# 1. *** FIX: Clear problematic environment variable for matplotlib ***
# This prevents the "ValueError: Key backend: 'module://matplotlib_inline.backend_inline'" error
if 'MPLBACKEND' in os.environ:
    del os.environ['MPLBACKEND']
    print("MPLBACKEND environment variable cleared.")

# 2. Clone the repository
!rm -rf FontDiffusion
!git clone https://github.com/dzungphieuluuky/FontDiffusion.git

Cloning into 'FontDiffusion'...
remote: Enumerating objects: 15148, done.
remote: Counting objects: 100% (2961/2961), done.
remote: Compressing objects: 100% (2868/2868), done.
remote: Total 15148 (delta 138), reused 2898 (delta 91), pack-reused 12187 (from 3)
Receiving objects: 100% (15148/15148), 247.43 MiB | 33.43 MiB/s, done.
Resolving deltas: 100% (526/526), done.
Updating files: 100% (128/128), done.


In [2]:
!uv pip install --upgrade pip
!uv pip install -r FontDiffusion/requirements.txt
!uv pip install gdown
# 3. Install PyTorch 1.13
print("\n⬇️ Installing PyTorch 1.13 (Required for this model)...")
# Force reinstall torch 1.13 to match the model's training environment
!uv pip uninstall torch torchvision
!uv pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117

# 4. Install other dependencies
print("\n⬇️ Installing Dependencies (Manually fixed)...")
# Install xformers compatible with Torch 1.13
!uv pip install xformers==0.0.16 -q

# Install original dependencies
!uv pip install transformers==4.33.1 accelerate==0.23.0 diffusers==0.22.0
!uv pip install gradio==4.8.0 pyyaml pygame opencv-python info-nce-pytorch kornia
# -----------------------------------------------------------------
!uv pip install lpips scikit-image pytorch-fid
!sudo apt-get update && sudo apt-get install dos2unix
print("\n✅ Environment setup complete. You can now proceed to Block 2 (Inference).")

Using Python 3.12.12 environment at: /usr
Resolved 1 package in 35ms
Audited 1 package in 0.20ms
Using Python 3.12.12 environment at: /usr
Resolved 116 packages in 201ms
  × Failed to build `tokenizers==0.13.3`
  ├─▶ The build backend returned an error
  ╰─▶ Call to `setuptools.build_meta.build_wheel` failed (exit status: 1)

      [stdout]
      running bdist_wheel
      running build
      running build_py
      copying py_src/tokenizers/__init__.py ->
      build/lib.linux-x86_64-cpython-312/tokenizers
      copying py_src/tokenizers/models/__init__.py ->
      build/lib.linux-x86_64-cpython-312/tokenizers/models
      copying py_src/tokenizers/decoders/__init__.py ->
      build/lib.linux-x86_64-cpython-312/tokenizers/decoders
      copying py_src/tokenizers/normalizers/__init__.py ->
      build/lib.linux-x86_64-cpython-312/tokenizers/normalizers
      copying py_src/tokenizers/pre_tokenizers/__init__.py ->
      build/lib.linux-x86_64-cpython-312/tokenizers/pre_tokenizers
      c

In [3]:
import os
from IPython import get_ipython

def configure_environment_paths():
    """Detect environment and configure paths"""
    try:
        if "google.colab" in str(get_ipython()):
            print("✅ Environment: Google Colab")
            base_data_path = "/content/"
            base_output_path = "/content/"
            environment_name = "colab"
        elif os.environ.get("KAGGLE_KERNEL_RUN_TYPE"):
            print("✅ Environment: Kaggle")
            base_data_path = "/kaggle/input/"
            base_output_path = "/kaggle/working/"
            environment_name = "kaggle"
        else:
            print("⚠️ Environment: Local/Unknown")
            base_data_path = "./data/"
            base_output_path = "./output/"
            environment_name = "local"
    except NameError:
        print("⚠️ Non-interactive session. Using local paths.")
        base_data_path = "./data/"
        base_output_path = "./output/"
        environment_name = "local"

    os.makedirs(base_output_path, exist_ok=True)
    print(f"📂 Data Path: {base_data_path}")
    print(f"📦 Output Path: {base_output_path}")

    return base_data_path, base_output_path, environment_name


INPUT_PATH, OUTPUT_PATH, ENV_NAME = configure_environment_paths()

✅ Environment: Google Colab
📂 Data Path: /content/
📦 Output Path: /content/


In [4]:
import os
import wandb

if "colab" in ENV_NAME:
    from google.colab import userdata

    try:
        # Ensure 'WANDB_API_KEY' is the exact name in your Colab Secrets (the key icon)
        wandb_key = userdata.get("WANDB_API_KEY")
        wandb.login(key=wandb_key)
    except Exception as e:
        print(f"Could not retrieve W&B API key from Colab Secrets: {e}")

# 2. Check if running in Kaggle
elif "kaggle" in ENV_NAME:
    try:
        from kaggle_secrets import UserSecretsClient

        user_secrets = UserSecretsClient()
        wandb_key = user_secrets.get_secret("WANDB_API_KEY")
        wandb.login(key=wandb_key)
    except Exception as e:
        print(f"Could not retrieve W&B API key from Kaggle Secrets: {e}")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dungngocpham171 (dungngocpham171-university-of-science) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [5]:
import gdown

if not os.path.exists("ckpt"):
  url = "https://drive.google.com/drive/folders/12hfuZ9MQvXqcteNuz7JQ2B_mUcTr-5jZ"
  gdown.download_folder(url, quiet=True, use_cookies=False)

In [6]:
# @title Unzipping all archived files
import os
import glob
from zipfile import ZipFile

zip_file_paths = glob.glob(os.path.join(INPUT_PATH, '*.zip'))

if not zip_file_paths:
    print(f'No .zip files found in {INPUT_PATH}.')
else:
    for zip_file_path in zip_file_paths:
        if os.path.exists(zip_file_path):
            print(f'Unzipping {zip_file_path}...')
            !unzip -q -o {zip_file_path} -d ./
            print(f'Unzipping of {zip_file_path} complete.')
        else:
            print(f'Error: The file {zip_file_path} was not found (post-glob check).')

No .zip files found in /content/.


In [7]:
# @title Checking checkpoint files (.pth)
import os
import time

CHECKPOINT_DIR = os.path.join(INPUT_PATH, "ckpt")
print(CHECKPOINT_DIR)
# Create the checkpoint directory
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
# Wait loop to check if files exist
required_files = ["unet.pth", "content_encoder.pth", "style_encoder.pth"]

while True:
    missing = [f for f in required_files if not os.path.exists(f"{CHECKPOINT_DIR}/{f}")]

    if not missing:
        print("\n✅ All weights found! You can proceed to the next step.")
        break
    else:
        print(f"Waiting for files... Missing: {missing}")
        print("Upload them to the 'ckpt' folder now.")
        time.sleep(10) # Checks every 10 seconds

/content/ckpt

✅ All weights found! You can proceed to the next step.


In [8]:
import pandas as pd
import os

def convert_csv_to_chars_txt(input_csv_path: str, output_txt_path: str, column_name: str = 'word'):
    """
    Reads a CSV file, extracts text from a specified column, and writes each character
    to a new line in a plain text file.

    Args:
        input_csv_path (str): The full path to the input CSV file.
        output_txt_path (str): The full path for the output text file.
        column_name (str): The name of the column in the CSV file containing the text.
    """
    if not os.path.exists(input_csv_path):
        print(f"Error: Input CSV file not found at '{input_csv_path}'. Please ensure the file is uploaded.")
        return

    try:
        df = pd.read_csv(input_csv_path)
    except Exception as e:
        print(f"Error reading CSV file '{input_csv_path}': {e}")
        return

    if column_name not in df.columns:
        print(f"Error: Column '{column_name}' not found in the CSV file '{input_csv_path}'.")
        return

    all_characters = []
    # Ensure the column values are treated as strings before iterating over them
    for item in df[column_name].astype(str).dropna().tolist():
        for char in item:
            all_characters.append(char)

    # Ensure output directory exists
    os.makedirs(os.path.dirname(output_txt_path), exist_ok=True)

    with open(output_txt_path, "w", encoding="utf-8") as f:
        f.write("\n".join(all_characters))
    print(f"Successfully converted '{input_csv_path}' to '{output_txt_path}', with one character per line.")

# --- Example Usage (demonstration with a dummy file) ---
# As the original file 'Ds_300_ChuNom_TuTao.csv' was not found in the previous execution,
# let's create a dummy file to demonstrate the function's usage.
print("\n--- Demonstrating function with a dummy CSV file ---")
dummy_csv_path = os.path.join(INPUT_PATH, "dummy_data.csv")
dummy_output_txt_path = os.path.join(OUTPUT_PATH, "dummy_chars.txt")

# Create a dummy CSV file
dummy_data = {'word': ['hello', 'world', 'python']}
pd.DataFrame(dummy_data).to_csv(dummy_csv_path, index=False)
print(f"Created a dummy CSV file at: {dummy_csv_path}")

convert_csv_to_chars_txt(dummy_csv_path, dummy_output_txt_path)

# --- How to use with your actual file ---
# Uncomment the lines below and replace 'your_actual_file.csv' and 'your_output.txt'
# with the correct paths for your use case.
#
# original_csv_file = os.path.join(INPUT_PATH, "Ds_300_ChuNom_TuTao.csv") # Or the full path to your CSV
# original_output_txt = os.path.join(OUTPUT_PATH, "nom_tu_tao.txt") # Or your desired output path
# convert_csv_to_chars_txt(original_csv_file, original_output_txt)



--- Demonstrating function with a dummy CSV file ---
Created a dummy CSV file at: /content/dummy_data.csv
Successfully converted '/content/dummy_data.csv' to '/content/dummy_chars.txt', with one character per line.


In [18]:
%cd {OUTPUT_PATH}/FontDiffusion
!python sample_batch.py \
    --characters "NomTuTao/Ds_10k_ChuNom_TuTao.txt" \
    --start_line 201 \
    --end_line 500 \
    --style_images "/content/FontDiffusion/styles_images" \
    --ckpt_dir "../ckpt/" \
    --ttf_path "fonts/NomNaTong-Regular.otf" \
    --output_dir "../my_dataset" \
    --batch_size 24 \
    --save_interval 5 \
    --channels_last \
    --num_inference_steps 20 \
    --guidance_scale 7.5 \
    --seed 42 \
    --compile \
    --enable_xformers

/content/FontDiffusion
pygame 2.6.1 (SDL 2.28.4, Python 3.12.12)
Hello from the pygame community. https://www.pygame.org/contribute.html

FONTDIFFUSER STANDARD FORMAT GENERATION
Loading characters from lines 201 to 500 (total: 10174 lines)
Successfully loaded 300 single characters.

Initializing font manager...
error: XDG_RUNTIME_DIR not set in the environment.
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_

In [22]:
import os
import glob
import zipfile
from typing import List
def find_result_folders(base_path: str) -> List[str]:
    """
    Return a list of absolute paths to all directories under `base_path`
    whose names start with 'results_'.
    """
    pattern = os.path.join(base_path, "*dataset")
    # glob returns both files and directories; filter to directories only
    return [p for p in glob.glob(pattern) if os.path.isdir(p)]
def zip_folder(folder_path: str, output_base_path: str) -> bool:
    """
    Zip the contents of `folder_path` into a file named
    <folder_name>.zip` inside `output_base_path`.

    Returns True on success, False otherwise.
    """
    folder_name = os.path.basename(folder_path)
    zip_path = os.path.join(output_base_path, f"{folder_name}.zip")
    try:
        print(f"   -> Zipping folder: {folder_name}...")
        with zipfile.ZipFile(
            zip_path, mode="w", compression=zipfile.ZIP_DEFLATED
        ) as zipf:
            for root, _, files in os.walk(folder_path):
                for file in files:
                    full_path = os.path.join(root, file)
                    # Preserve relative path inside the zip
                    arcname = os.path.relpath(full_path, os.path.dirname(folder_path))
                    zipf.write(full_path, arcname)
        print(f"   ✅ Created ZIP: {os.path.basename(zip_path)}")
        return True
    except Exception as exc:
        print(f"   ❌ Failed to zip {folder_name}: {exc}")
        return False
def zip_stats_results_folders(output_base_path: str) -> None:
    """
    Main driver: locate all result folders and zip each one.
    """
    # Ensure the output directory exists
    os.makedirs(output_base_path, exist_ok=True)
    result_folders = find_result_folders(output_base_path)
    if not result_folders:
        print(f"⚠️ No folders starting with 'results_' found in '{output_base_path}'.")
        return
    print(f"🔍 Found {len(result_folders)} result folder(s) to zip.")
    successful = 0
    for folder in result_folders:
        if zip_folder(folder, output_base_path):
            successful += 1
    print(
        f"\n✅ DONE! Successfully zipped {successful} out of {len(result_folders)} folder(s)."
    )
if __name__ == "__main__":
    try:
        # Prefer an environment variable; fall back to a global if defined
        output_root = os.getenv("OUTPUT_PATH") or globals().get("OUTPUT_PATH")
        if not output_root:
            raise ValueError("OUTPUT_PATH not defined")
        # The script expects a sub‑folder named 'OuroTrace' under OUTPUT_PATH
        target_path = os.path.join(output_root, "")
        zip_stats_results_folders(target_path)
    except Exception as e:
        print(f"❌ An error occurred: {e}")


🔍 Found 1 result folder(s) to zip.
   -> Zipping folder: my_dataset...
   ✅ Created ZIP: my_dataset.zip

✅ DONE! Successfully zipped 1 out of 1 folder(s).


In [23]:
# @title Happy Christmas✨
# !rm -r -f FontDiffusion